In [54]:
from typing import List, Tuple, Optional
from transformers import AutoTokenizer

In [57]:
MODEL_PATH = ""
LLMS: List[Tuple[str, str, Optional[str]]] = [
    # Meta
    ("meta-llama/Llama-3.1-8B-Instruct", MODEL_PATH + "meta-llama/Llama-3.1-8B-Instruct", None),
    ("hugging-quants/Meta-Llama-3.1-70B-Instruct-AWQ-INT4", MODEL_PATH + "hugging-quants/Meta-Llama-3.1-70B-Instruct-AWQ-INT4", "awq"),
    ("meta-llama/Llama-3.2-3B-Instruct", MODEL_PATH + "meta-llama/Llama-3.2-3B-Instruct", None),
    ("ibnzterrell/Meta-Llama-3.3-70B-Instruct-AWQ-INT4", MODEL_PATH + "ibnzterrell/Meta-Llama-3.3-70B-Instruct-AWQ-INT4", "awq"),
    
    # Microsoft
    ("microsoft/Phi-3-mini-128k-instruct", MODEL_PATH + "microsoft/Phi-3-mini-128k-instruct", None),
    ("microsoft/Phi-3-small-128k-instruct", MODEL_PATH + "microsoft/Phi-3-small-128k-instruct", None),
    ("microsoft/Phi-3-medium-128k-instruct", MODEL_PATH + "microsoft/Phi-3-medium-128k-instruct", None),
    ("microsoft/Phi-3.5-mini-instruct", MODEL_PATH + "microsoft/Phi-3.5-mini-instruct", None),
    ("microsoft/Phi-4-mini-instruct", MODEL_PATH + "microsoft/Phi-4-mini-instruct", None),
    ("microsoft/phi-4", MODEL_PATH + "microsoft/phi-4", None),
    
    # Mistral
    ("mistralai/Mistral-Nemo-Instruct-2407", MODEL_PATH + "mistralai/Mistral-Nemo-Instruct-2407", None),
    ("mistralai/Ministral-8B-Instruct-2410", MODEL_PATH + "mistralai/Ministral-8B-Instruct-2410", None),
    
    # Qwen
    ("Qwen/Qwen2-72B-Instruct-AWQ", MODEL_PATH + "Qwen/Qwen2-72B-Instruct-AWQ", "awq"),
    ("Qwen/Qwen2-7B-Instruct", MODEL_PATH + "Qwen/Qwen2-7B-Instruct", None),
    ("Qwen/Qwen2.5-72B-Instruct-AWQ", MODEL_PATH + "Qwen/Qwen2.5-72B-Instruct-AWQ", "awq"),
    ("Qwen/Qwen2.5-14B-Instruct", MODEL_PATH + "Qwen/Qwen2.5-14B-Instruct", None),
    ("Qwen/Qwen2.5-7B-Instruct", MODEL_PATH + "Qwen/Qwen2.5-7B-Instruct", None),
    ("Qwen/Qwen2.5-3B-Instruct", MODEL_PATH + "Qwen/Qwen2.5-3B-Instruct", None),
    
    # Falcon
    ("tiiuae/Falcon3-7B-Instruct", MODEL_PATH + "tiiuae/Falcon3-7B-Instruct", None),
    ("tiiuae/Falcon3-3B-Instruct", MODEL_PATH + "tiiuae/Falcon3-3B-Instruct", None),
]

In [61]:
for model, path, quant in  LLMS:
    print(model)
    tokenizer = AutoTokenizer.from_pretrained(path)
    print(tokenizer("Hello, how are you?")["input_ids"], len(tokenizer("Hello, how are you?")["input_ids"]))

meta-llama/Llama-3.1-8B-Instruct
[128000, 9906, 11, 1268, 527, 499, 30] 7
hugging-quants/Meta-Llama-3.1-70B-Instruct-AWQ-INT4
[128000, 9906, 11, 1268, 527, 499, 30] 7
meta-llama/Llama-3.2-3B-Instruct
[128000, 9906, 11, 1268, 527, 499, 30] 7
ibnzterrell/Meta-Llama-3.3-70B-Instruct-AWQ-INT4
[128000, 9906, 11, 1268, 527, 499, 30] 7
microsoft/Phi-3-mini-128k-instruct
[15043, 29892, 920, 526, 366, 29973] 6
microsoft/Phi-3-small-128k-instruct
[9906, 11, 1268, 527, 499, 30] 6
microsoft/Phi-3-medium-128k-instruct
[15043, 29892, 920, 526, 366, 29973] 6
microsoft/Phi-3.5-mini-instruct
[15043, 29892, 920, 526, 366, 29973] 6
microsoft/Phi-4-mini-instruct
[13225, 11, 1495, 553, 481, 30] 6
microsoft/phi-4
[9906, 11, 1268, 527, 499, 30] 6
mistralai/Mistral-Nemo-Instruct-2407
[1, 22177, 1044, 2606, 1584, 1636, 1063] 7
mistralai/Ministral-8B-Instruct-2410
[1, 22177, 1044, 2606, 1584, 1636, 1063] 7
Qwen/Qwen2-72B-Instruct-AWQ
[9707, 11, 1246, 525, 498, 30] 6
Qwen/Qwen2-7B-Instruct
[9707, 11, 1246, 525, 